In [11]:
import sqlite3
import random
import requests
from bs4 import BeautifulSoup
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import AzureChatOpenAI
from datetime import datetime
from langchain.schema import BaseMessage
import networkx as nx
import matplotlib.pyplot as plt

# StoreSupervisor Decision-Making Process
class StoreSupervisor:
    def __init__(self):
        self.approval_policy = "Approval policy based on performance"
        self.rejection_policy = "Rejection policy based on performance"

    def invoke(self, input_payload):
        """
        Simulate decision-making process based on store goals and staff performance
        """
        staff_feedback = input_payload["staff_feedback"]
        store_name = input_payload["store_name"]
        
        if "exceeding" in staff_feedback:
            return "Approved"
        elif "struggling" in staff_feedback:
            return "Rejected"
        return "Reviewed"

# SQLite Database
def create_store_tables():
    connection = sqlite3.connect("store_targets.db")
    cursor = connection.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS store_targets (
                        id INTEGER PRIMARY KEY,
                        store_name TEXT NOT NULL,
                        target INTEGER NOT NULL,
                        insight TEXT
                    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS staff_status (
                        staff_id INTEGER PRIMARY KEY,
                        staff_name TEXT NOT NULL,
                        store_name TEXT NOT NULL,
                        progress INTEGER NOT NULL,
                        feedback TEXT,
                        motivated BOOLEAN,
                        supervisor_id INTEGER
                    )''')
    connection.commit()
    connection.close()

# Alter staff_status table to add supervisor_id if missing
def alter_staff_table():
    connection = sqlite3.connect("store_targets.db")
    cursor = connection.cursor()
    try:
        cursor.execute('''ALTER TABLE staff_status ADD COLUMN supervisor_id INTEGER''')
    except sqlite3.OperationalError:
        print("Column 'supervisor_id' already exists.")
    connection.commit()
    connection.close()

# Store Staff Data
def store_staff_data(staff_data):
    connection = sqlite3.connect("store_targets.db")
    cursor = connection.cursor()
    cursor.executemany('''INSERT OR REPLACE INTO staff_status
                           (staff_id, staff_name, store_name, progress, feedback, motivated, supervisor_id)
                           VALUES (?, ?, ?, ?, ?, ?, ?)''',
                           [(member["staff_id"], member["staff_name"], member["store_name"],
                             member["progress"], member["feedback"],
                             member["motivated"], member["supervisor_id"])
                            for member in staff_data])
    connection.commit()
    connection.close()

# Azure Chat Model Setup
store_feedback_model = AzureChatOpenAI(
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    model="gpt-4o-mini",
    azure_endpoint="https://xxxx.openai.azure.com/",
    azure_deployment="gpt-4o-mini",
    api_version="2024-08-01-preview",
    temperature=0.7
)

# Generate AI feedback
def generate_ai_feedback(staff_feedback, progress_value):
    prompt = (
        f"Staff Feedback: {staff_feedback}\n"
        f"Progress Report: {progress_value}%\n\n"
        "Provide constructive and detailed feedback for this staff member."
    )

    messages = [
        BaseMessage(type="user", content=prompt)
    ]

    try:
        response = store_feedback_model.generate(
            messages=messages,
            max_tokens=200,
            temperature=0.7,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )

        if 'choices' in response and len(response['choices']) > 0:
            return response['choices'][0]['message']['content'].strip()
        else:
            return "No valid feedback generated."
    except Exception as error:
        print(f"AI feedback error: {error}")
        return "Error generating feedback."

# Simulate Feedback and Store Target Adjustment
def simulate_feedback_and_target_update(staff_id, staff_feedback, store_name, progress_value):
    connection = sqlite3.connect("store_targets.db")
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM staff_status WHERE staff_id=?", (staff_id,))
    staff_member = cursor.fetchone()

    if not staff_member:
        return "Staff member not found."

    cursor.execute(
        "UPDATE staff_status SET progress=?, feedback=?, motivated=? WHERE staff_id=?",
        (progress_value, staff_feedback, progress_value >= 80, staff_id)
    )
    connection.commit()

    print(f"Updated progress for {staff_member[1]} to {progress_value}%")

    ai_feedback = generate_ai_feedback(staff_feedback, progress_value)

    supervisor = StoreSupervisor()
    decision = supervisor.invoke({
        "staff_feedback": staff_feedback,
        "store_name": store_name
    })

    current_target = 85
    if decision == "Approved":
        new_target = current_target + 5
    elif decision == "Rejected":
        new_target = current_target - 5
    else:
        new_target = current_target

    cursor.execute('''INSERT OR REPLACE INTO store_targets (store_name, target, insight)
                      VALUES (?, ?, ?)''',
                   (store_name, new_target, "Adjusted based on supervisor decision"))
    connection.commit()
    connection.close()

    return (
        f"Staff {staff_member[1]}'s progress updated to {progress_value}%. "
        f"Decision: {decision}. Store target updated to {new_target}%. "
        f"AI Feedback: {ai_feedback}"
    )

# Build Store Hierarchy Graph
def build_store_hierarchy():
    graph = nx.DiGraph()
    connection = sqlite3.connect("store_targets.db")
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM staff_status")
    rows = cursor.fetchall()

    for row in rows:
        staff_id = row[0]
        staff_name = row[1]
        supervisor_id = row[6]
        graph.add_node(staff_id, label=f"{staff_name} ({staff_id})", store=row[2])
        if supervisor_id:
            graph.add_edge(supervisor_id, staff_id)

    return graph

# Visualize Store Hierarchy
def visualize_store_hierarchy():
    graph = build_store_hierarchy()
    positions = nx.circular_layout(graph)
    labels = nx.get_node_attributes(graph, "label")

    plt.figure(figsize=(6, 4))
    nx.draw(
        graph, positions, with_labels=True,
        node_size=1200, node_color="lightgreen",
        font_size=6, font_weight="bold",
        arrows=True, edge_color="gray"
    )
    nx.draw_networkx_labels(graph, positions, labels, font_size=8)
    plt.title("Store Staff Hierarchy", size=12)
    plt.show()

# Regional Management Agent
class RegionalManagerAgent:
    def __init__(self, store_name):
        self.store_name = store_name

    def report(self, target, report_text):
        print(f"Regional Report for {self.store_name}:")
        print(f"  - Store Target: {target}%")
        print(f"  - Report: {report_text}")

# Store Agent
class StoreAgent:
    def __init__(self, store_name, staff, central_agent):
        self.store_name = store_name
        self.staff = staff
        self.supervisor = StoreSupervisor()
        self.regional_manager = RegionalManagerAgent(store_name)
        self.central_agent = central_agent

    def handle_staff_feedback(self, staff_id, staff_feedback, progress_value):
        result = simulate_feedback_and_target_update(
            staff_id, staff_feedback, self.store_name, progress_value
        )

        self.regional_manager.report(85, result)
        self.central_agent.receive_report(self.store_name, result)
        self.send_staff_message(staff_id, staff_feedback)

    def send_staff_message(self, staff_id, staff_feedback):
        message = (
            f"Hello Staff {staff_id},\n\n"
            f"Feedback: {staff_feedback}\n\n"
            "Please review your progress and focus on improvements. "
            "Use the store app or Slack for quick updates.\n\nThank you."
        )
        print(f"Message sent to Staff {staff_id}: {message}")

        response = self.simulate_staff_response(staff_feedback)
        print(f"Staff Response: {response}")

        if "struggling" in staff_feedback:
            print("Recommended Channel: Phone or Slack")
        else:
            print("Recommended Channel: Slack or Email")

    def simulate_staff_response(self, staff_feedback):
        if "exceeding" in staff_feedback:
            return "Thanks! I'll keep delivering strong results."
        elif "struggling" in staff_feedback:
            return "Understood. I'll work on improving immediately."
        return "Thanks for the feedback."

# Central Operations Agent
class CentralOperationsAgent:
    def __init__(self):
        self.reports = {}

    def receive_report(self, store_name, report):
        if store_name not in self.reports:
            self.reports[store_name] = []
        self.reports[store_name].append(report)

    def print_reports(self):
        print("\n=== Aggregated Reports from All Stores ===")
        for store_name, reports in self.reports.items():
            print(f"\nStore: {store_name}")
            for report in reports:
                print(f"  - {report}")

# Run Simulation
def run_store_simulation():
    alter_staff_table()

    staff_data = [
        {"staff_id": 1, "staff_name": "Alice", "store_name": "Downtown", "progress": 70, "feedback": "", "motivated": False, "supervisor_id": None},
        {"staff_id": 2, "staff_name": "Bob", "store_name": "Uptown", "progress": 80, "feedback": "", "motivated": False, "supervisor_id": 1},
        {"staff_id": 3, "staff_name": "Charlie", "store_name": "Mall", "progress": 90, "feedback": "", "motivated": False, "supervisor_id": 1},
        {"staff_id": 4, "staff_name": "David", "store_name": "Outlet", "progress": 60, "feedback": "", "motivated": False, "supervisor_id": 2},
    ]

    store_staff_data(staff_data)

    central_agent = CentralOperationsAgent()

    downtown_agent = StoreAgent("Downtown", staff_data[:1], central_agent)
    uptown_agent = StoreAgent("Uptown", staff_data[1:2], central_agent)
    mall_agent = StoreAgent("Mall", staff_data[2:3], central_agent)
    outlet_agent = StoreAgent("Outlet", staff_data[3:], central_agent)

    print("\n SIMULATING STORE FEEDBACK...\n")
    downtown_agent.handle_staff_feedback(1, "Great progress! You're exceeding expectations.", 85)
    uptown_agent.handle_staff_feedback(2, "Struggling to meet targets. Please improve.", 75)
    mall_agent.handle_staff_feedback(3, "Exceeding expectations in sales. Keep it up!", 95)
    outlet_agent.handle_staff_feedback(4, "Struggling with store operations. Needs improvement.", 65)

    central_agent.print_reports()

if __name__ == "__main__":
    create_store_tables()
    run_store_simulation()
    visualize_store_hierarchy()


In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

def show_store_performance_analytics():
    # Connect and load data
    connection = sqlite3.connect("store_targets.db")
    dataframe = pd.read_sql_query("SELECT * FROM staff_status", connection)
    connection.close()

    # Grouped summary per store
    store_summary = (
        dataframe
        .groupby("store_name")["progress"]
        .agg(["mean", "min", "max", "count"])
        .reset_index()
    )
    store_summary.columns = [
        "Store",
        "Average Progress",
        "Lowest",
        "Highest",
        "Staff Count"
    ]

    # Display performance summary
    print("=== Store Performance Summary ===")
    display(
        store_summary.style
        .set_caption("Performance Summary by Store")
        .background_gradient(cmap='Greens')
        .set_table_styles([
            {'selector': 'thead th', 'props': [('background-color', '#3b7a57'), ('color', 'white')]},
            {'selector': 'tbody td', 'props': [('font-size', '12pt'), ('padding', '8px')]},
            {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]}
        ])
        .format({
            "Average Progress": "{:.1f}%",
            "Lowest": "{:.0f}%",
            "Highest": "{:.0f}%"
        })
    )

    # Visual: Average Progress per Store
    plt.figure(figsize=(10, 6))
    sns.set(style="whitegrid", palette="coolwarm")
    axis = sns.barplot(
        data=store_summary,
        x="Store",
        y="Average Progress",
        palette="viridis"
    )

    axis.set_title(
        "Average Progress by Store",
        fontsize=16,
        fontweight='bold',
        color='darkgreen'
    )
    axis.set_xlabel("Store", fontsize=12)
    axis.set_ylabel("Progress (%)", fontsize=12)
    axis.set_xticklabels(
        axis.get_xticklabels(),
        rotation=45,
        ha="right",
        fontsize=11
    )

    axis.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Highlight underperforming staff (below 70%)
    low_performance_staff = dataframe[dataframe["progress"] < 70]
    if not low_performance_staff.empty:
        print("\n Staff Members Below 70% Progress:")
        display(
            low_performance_staff[
                ["staff_id", "staff_name", "store_name", "progress"]
            ]
        )

# Call analytics function
show_store_performance_analytics()
